In [ ]:
import gurobipy as gp
import pandas as pd

# Data

In [143]:
categories = ["shoes", "gloves", "clubheads", "outerwear"]
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
demand_calendar = [[350,330,140,60,0, 80, 140, 200, 220, 220, 230, 240],
          [70, 100, 20, 0, 0, 50, 70, 90, 100, 110, 115, 100],
          [18, 16, 10, 8, 5, 3, 5, 15, 25, 30, 30, 30],
          [12, 10, 8, 8, 6, 3, 3, 30, 70, 70, 80, 80]
]

nMonths = 12
range_months = range(nMonths)
range_categories = range(len(categories))

demand=[[0 for j in range_months] for i in range_categories]

# Shift by 7, since we start in June
for i in range_categories:
    demand[i] = demand_calendar[i][-7:] + demand_calendar[i][:-7]

duty_rates = [0.19, 0.13, 0.05, 0.185]
storage_costs = [0.36, 0.22, 0.44, 0.32]
unit_values = [40, 7, 88, 60]
units_per_carton = [12, 336, 25, 42]
cartons_per_pallet = [20, 15, 48, 15]
totals_per_pallet = [unit_values[i] * units_per_carton[i] * cartons_per_pallet[i] for i in range_categories]

pallets = []  # 1000's of them (sum of all demand)
for i in range_categories:
    for j in range(sum(demand[i])):
        pallets.append(i)
range_pallets = range(len(pallets))

# monthly rate of return for investments of money saved by delaying duty rate
r = 0.13/12

# Variables

In [144]:
# Model
ACI_Model = gp.Model("ACI_Model")
# Variables

# Month Variables
# ccbw_in_p is the month that pallet p is first put into storage at
ccbw_in = ACI_Model.addVars(range_pallets, vtype = gp.GRB.INTEGER, name="ccbw_in")
# ccbw_out_p is the month that pallet p is taken out of storage. Investment profit is not made for this month.
ccbw_out = ACI_Model.addVars(range_pallets, vtype = gp.GRB.INTEGER, name="ccbw_out")

# Binary Variables
# x_pm is 1 if pallet p is inside of CCBW storage at month m, 0 otherwise
x = ACI_Model.addVars(range_pallets, range_months, vtype = gp.GRB.BINARY, name = "x")
# y_pm is 1 if m >= ccbw_in_p, 0 otherwise
y = ACI_Model.addVars(range_pallets, range_months, vtype = gp.GRB.BINARY, name = "y")
# z_pm is 1 if m <= ccbw_out_p, 0 otherwise
z = ACI_Model.addVars(range_pallets, range_months, vtype = gp.GRB.BINARY, name = "z")
# a_pm is 1 if ccbw_in_p <=m <= ccbw_out_p
a = ACI_Model.addVars(range_pallets, range_months, vtype = gp.GRB.BINARY, name = "a")

# Constraints

In [145]:
# Maximum 200 pallets at the ccbw at a given month
for m in range_months:
    ACI_Model.addConstr(gp.quicksum(x[p,m] for p in range_pallets) <= 200)

# the sum of pallets going into CCBW is less than the demand (exhibit 2)
for m in range_months:
    for c in range_categories:
        ACI_Model.addConstr(demand[c][m] - gp.quicksum(x[p,m] for p in range_pallets if pallets[p] == c) >= 0)
        
######## Constraints to prevent pallets being stored in non-consecutive months ########
M = 1000
for p in range_pallets:
    for m in range_months:
        ACI_Model.addConstr(y[p,m] * M >= m - ccbw_in[p] + 1)
        ACI_Model.addConstr((y[p,m] - 1) * M <= m - ccbw_in[p])

        ACI_Model.addConstr(z[p,m] * M >= ccbw_out[p] - m)
        ACI_Model.addConstr((z[p,m] - 1) * M <= ccbw_out[p] - m - 1)

        ACI_Model.addConstr(a[p,m] == y[p,m] + z[p,m] - 1)

        ACI_Model.addConstr(x[p,m] <= a[p,m])

# Constraints to constrain ccbw_in and ccbw_out
for p in range_pallets:
    # the month that the pallet goes into the ccbw is before the month that the pallet exits the ccbw
    ACI_Model.addConstr(ccbw_in[p] <= ccbw_out[p])
    # the month that the pallet goes into the ccbw must be less than 12
    ACI_Model.addConstr(ccbw_in[p] <= len(range_months))
    # the month that the pallet exits the ccbw must be less than 12
    ACI_Model.addConstr(ccbw_out[p] <= len(range_months))

# A pallet is in the ccbw for as many months as its stay, which is defined by ccbw_out_p - ccbw_in_p. This links the 3 variables together.
for p in range_pallets:
    ACI_Model.addConstr(ccbw_out[p] - ccbw_in[p] == gp.quicksum(x[p,m] for m in range_months))

# Objective Function

In [146]:
# profit is how much ACI makes from investing money they have by delaying duty rate
profit=0
for c in range_categories:
    profit += gp.quicksum(x[p,m] for p in range_pallets for m in range_months if pallets[p] == c) * duty_rates[c] * r * totals_per_pallet[c]

# Monthly storage costs at CCBW + duty rate when they procure products from CCBW
expenses = 0
for c in range_categories:
    expenses += gp.quicksum(x[p,m] for p in range_pallets for m in range_months if pallets[p] == c) * storage_costs[c] * cartons_per_pallet[c] 
    
revenue = profit - expenses
ACI_Model.setObjective(revenue, gp.GRB.MAXIMIZE)
ACI_Model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 274420 rows, 180500 columns and 714780 nonzeros
Model fingerprint: 0x2f24332e
Variable types: 0 continuous, 180500 integer (173280 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 54160 rows and 47180 columns
Presolve time: 1.27s
Presolved: 220260 rows, 133320 columns, 582790 nonzeros
Variable types: 0 continuous, 133320 integer (126100 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.487e+03
 Factor NZ  : 9.6

# Print Solution

## Objective Value

In [154]:
print(f'The maximum revenue that can be made is: ${ACI_Model.objVal}.')
#78758.06000000025

The maximum revenue that can be made is: $78758.06000000023.


## CCBW Storage Schedule

In [148]:
df = pd.DataFrame(index=categories, columns=months)
for c in range_categories:
    for m in range_months:
        df.iloc[c,m] = gp.quicksum(x[p,m].x for p in range_pallets if pallets[p] == c)
df.loc['sum'] = df.sum()
df

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
shoes,80.0,122.0,65.0,5.0,0.0,0.0,0.0,100.0,74.0,140.0,60.0,0.0
gloves,50.0,70.0,90.0,100.0,110.0,115.0,100.0,70.0,100.0,20.0,0.0,0.0
clubheads,3.0,5.0,15.0,25.0,20.0,5.0,20.0,18.0,16.0,10.0,8.0,5.0
outerwear,3.0,3.0,30.0,70.0,70.0,80.0,80.0,12.0,10.0,8.0,8.0,6.0
sum,136.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,178.0,76.0,11.0


## Specific Instructions for Each Pallet

In [153]:
for p in range_pallets:
    pallet_months = []
    for m in range_months:
        if x[p,m].x != 0:
            pallet_months.append(m)
        if m == range_months[-1]:
            if len(pallet_months) >= 1:
                print(f'Store pallet ID: {p} during months {pallet_months}')

Store pallet ID: 13 during months [0, 1, 2]
Store pallet ID: 15 during months [2]
Store pallet ID: 21 during months [8, 9, 10]
Store pallet ID: 26 during months [0, 1, 2]
Store pallet ID: 27 during months [0, 1]
Store pallet ID: 30 during months [9, 10]
Store pallet ID: 33 during months [7, 8, 9, 10]
Store pallet ID: 34 during months [7, 8, 9]
Store pallet ID: 35 during months [0, 1, 2]
Store pallet ID: 40 during months [0, 1, 2]
Store pallet ID: 41 during months [7]
Store pallet ID: 44 during months [0, 1, 2]
Store pallet ID: 48 during months [7, 8, 9]
Store pallet ID: 50 during months [7, 8, 9]
Store pallet ID: 54 during months [9, 10]
Store pallet ID: 56 during months [7, 8, 9]
Store pallet ID: 57 during months [7]
Store pallet ID: 63 during months [1]
Store pallet ID: 65 during months [9, 10]
Store pallet ID: 68 during months [7, 8]
Store pallet ID: 76 during months [7]
Store pallet ID: 79 during months [1]
Store pallet ID: 82 during months [9]
Store pallet ID: 96 during months [9,

# Ensure that each pallet is stored during consecutive months

In [152]:
def has_consecutive_numbers(lst):
    # Sort the list
    lst_sorted = sorted(lst)
    
    # Check if each element is one more than the previous one
    for i in range(1, len(lst_sorted)):
        if lst_sorted[i] != lst_sorted[i - 1] + 1:
            return False
    return True
# if nothing is printed, it means that there are no pallets that are stored at non-consecutive months
for p in range_pallets:
    pallet_months = []
    for m in range_months:
        if x[p,m].x != 0:
            pallet_months.append(m)
        if m == range_months[-1]:
            if has_consecutive_numbers(pallet_months) and len(pallet_months) >= 5 :
                pass
                #print(f'Category {pallets[p]}: {pallet_months}')
            elif not has_consecutive_numbers(pallet_months):
                
                print(f"FAIL: Category {pallets[p]}: {pallet_months}")